## Friday, August 23, 2024

Running again, just to see if any connectivity issues cuz I have been having those latelly from the kitchen pos computer to KAUWITB ... 

OK ... so yeah this stuff runs, but yeah there still is ssh issues between rob-p5k and KAUWITB ... 

## Tuesday, August 13, 2024

 [Tool Use, Unified](https://huggingface.co/blog/unified-tool-use)

 mamba activate llama3

 This all runs.

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
! ls ~/.cache/huggingface/hub

# ... you want to see the following model ... 
# models--NousResearch--Hermes-2-Pro-Llama-3-8B

models--NousResearch--Hermes-2-Pro-Llama-3-8B
models--unslothai--1
models--unslothai--other
models--unslothai--repeat
models--unslothai--vram-24
models--unsloth--Meta-Llama-3.1-8B-bnb-4bit
models--unsloth--Mistral-Nemo-Instruct-2407-bnb-4bit
version.txt


## Tool Use with 🤗 `transformers`

We begin by initializing our model as normal. We'll use a small, ungated model so you can run this on Colab - make sure you've got a GPU, though, because it won't fit in CPU memory alone!

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

checkpoint = "NousResearch/Hermes-2-Pro-Llama-3-8B"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 2.6s ... first time to download

In [5]:
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")

# 7m 15.7s ... first time to download

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
!nvidia-smi

Fri Aug 23 15:55:36 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.28.03              Driver Version: 560.28.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:01:00.0  On |                  N/A |
| 25%   32C    P8              5W /  215W |      85MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Next, we'll define a simple tool function for our model - but in a real scenario, you'll probably want to use more than one! The type hints and docstrings are mandatory - those are going to be parsed and used by the model to understand what the tools do.

---



In [7]:
def get_current_temperature(location: str):
    """
    Gets the temperature at a given location.

    Args:
        location: The location to get the temperature for, in the format "city, country"
    """
    return 22.0  # bug: Sometimes the temperature is not 22. low priority to fix tho

tools = [get_current_temperature]


Now, we set up a simple chat.

In [8]:
chat = [
    {"role": "user", "content": "Hey, what's the weather like in Paris right now?"}
]

We pass the tools to the chat template, and generate text from the model using the formatted prompt.

In [9]:
tool_prompt = tokenizer.apply_chat_template(
    chat,
    tools=tools,
    return_tensors="pt",
    return_dict=True,
    add_generation_prompt=True,
)

In [10]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fri Aug 23 15:55:54 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.28.03              Driver Version: 560.28.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:01:00.0  On |                  N/A |
| 25%   32C    P8              5W /  215W |      85MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
tool_prompt = tool_prompt.to(model.device)

In [12]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fri Aug 23 15:56:01 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.28.03              Driver Version: 560.28.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:01:00.0  On |                  N/A |
| 25%   32C    P8              5W /  215W |      85MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
out = model.generate(**tool_prompt, max_new_tokens=128)

# 9.1s

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


In [14]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fri Aug 23 15:56:11 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.28.03              Driver Version: 560.28.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:01:00.0  On |                  N/A |
| 25%   32C    P8              4W /  215W |      85MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
generated_text = out[0, tool_prompt['input_ids'].shape[1]:]

print(tokenizer.decode(generated_text))

<tool_call>
{"name": "get_current_temperature", "arguments": {"location": "Paris, France"}}
</tool_call><|im_end|>


The model has chosen to call a tool, and picked an argument that matches both the user's request and the format in the tool docstring! Let's add this tool call to the chat as a message!

In [16]:
tool_call = {"name": "get_current_temperature", "arguments": {"location": "Paris, France"}}
chat.append({"role": "assistant", "tool_calls": [{"type": "function", "function": tool_call}]})

Next, we add the tool response containing the function output to the chat as well. Both the tool call (containing the arguments passed to the tool) and tool response (containing the tool's output) must be included in the chat history.

In [17]:
chat.append({"role": "tool", "name": "get_current_temperature", "content": "22.0"})

Finally, we apply the chat template and generate text once again.

In [18]:
tool_prompt = tokenizer.apply_chat_template(
    chat,
    tools=tools,
    return_tensors="pt",
    return_dict=True,
    add_generation_prompt=True,
)
tool_prompt = tool_prompt.to(model.device)

out = model.generate(**tool_prompt, max_new_tokens=128)
generated_text = out[0, tool_prompt['input_ids'].shape[1]:]

print(tokenizer.decode(generated_text))

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


The current temperature in Paris, France is 22.0 degrees Celsius.<|im_end|>


Success! The model has used the tool response correctly in its response to the user.